In [10]:
import talos as ta
import pandas as pd
import numpy as np

from talos.model.normalizers import lr_normalizer
from talos.model.early_stopper import early_stopper

from keras.models import Model, load_model
from keras.layers import Dense, Input, Concatenate, Lambda, regularizers, LeakyReLU, Softmax
from keras.optimizers import Adam, Nadam
from keras.callbacks import TensorBoard, ModelCheckpoint, EarlyStopping

from sklearn.model_selection import train_test_split

In [31]:
params = {
    "lr": (0.1, 10, 10),
    "batch_size": [64, 128, 256],
    "shared_weights": [[32, 32], [32, 16], [16, 16], [16, 8]],
    "fully_connected": [[256, 256, 128], [256, 128, 128], [128, 65, 64], [32, 32, 16]],
    "optimizer": [Adam],
    "epochs": [50]
}

In [32]:
def model_fit(x_train, y_train, x_val, y_val, params):
        inputArray = Input(shape=(24,))
        layer = inputArray

        shared_dense = Dense(params["shared_weights"][0], activation="relu")
        layerArray = []
        h = 6
        for t in range(4):
            out = Lambda(lambda x: x[:, t*h:(t+1)*h])(layer)
            layerArray.append(shared_dense(out))

        layer = layerArray

        for neurons in params["shared_weights"][1:]:
            shared_dense = Dense(neurons, activation='relu')
            layerArray = []
            for t in range(4):
                layerArray.append(shared_dense(layer[t]))
            layer = layerArray

        merged_vector = Concatenate(axis=-1)(layer)

        layer = merged_vector
        for neurons in params["fully_connected"]:
            layer = Dense(neurons, activation='relu')(layer)
        output_layer = Dense(1, activation='linear')(layer)
        model = Model(input=inputArray, output=output_layer)

        opt = params["optimizer"](lr_normalizer(params['lr'], params['optimizer']))
        model.compile(optimizer=opt, loss='mse', metrics=['mae'])
        
        
        history = model.fit(x_train, y_train, 
                    validation_data=[x_val, y_val],
                    batch_size=params['batch_size'],
                    callbacks=[early_stopper(params['epochs'], mode='strict')],
                    epochs=params['epochs'],
                    verbose=0)

        return history, model

In [9]:
df = pd.read_csv("up_to_8.csv")
df["StateRepresentation"] = df["StateRepresentation"].apply(lambda x: np.fromstring(x[1:-1], sep=" "))
df["MovesEncoded"] = df["MovesEncoded"].apply(lambda x: np.fromstring(x[1:-1], sep=" "))
df["hashed"] = df["StateRepresentation"].apply(lambda s: s.tostring())
df = df.sort_values('Value', ascending=False).drop_duplicates('hashed').sort_index()

In [12]:
max_val = 16
x_data = df.StateRepresentation.values
x_data = np.array([x / max_val for x in x_data])

y = df.Value.values
y = np.array([np.array([val], dtype=float) for val in y])

In [19]:
X_train, X_test, y_train, y_test = train_test_split(x_data, y, test_size=0.2, random_state=25)

In [34]:
h = ta.Scan(X_train, y_train, params=params,
            model=model_fit,
            dataset_name="4x6",
            experiment_no='2',
            grid_downsample=.01)







  0%|          | 0/15 [00:00<?, ?it/s]





  7%|▋         | 1/15 [01:30<21:02, 90.15s/it]

KeyboardInterrupt: 